In [130]:
from selenium import webdriver
from selenium.webdriver.ie.options import Options

# chromedriver
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
driver = webdriver.Chrome(options=options)

- 떠있는 크롬 화면에서 로그인 필요
- 구글 로그인 해야 해서 로그인을 시스템으로 만드려면 또 복잡...

## link에서 api 리스트 정보 수집

In [131]:
# Load Page
driver.get(url='https://rapidapi.com/collection/list-of-free-apis')

In [132]:
from selenium.webdriver.common.by import By


def pagenation(driver, trial):
    pages = driver.find_elements(By.XPATH, f"//li[contains(@class, 'r-page-item')]")
    idx = -1
    for i, page in enumerate(pages):
        _class = page.get_attribute("class")
        if _class.endswith("selected"):
            idx = i
    if -1 < idx and idx + 1 < len(pages):
        button = pages[idx + 1].find_element(By.XPATH, ".//button")
        print(f"page {trial}")
        disabled = button.get_attribute("disabled")
        if not disabled:
            button.click()
    return idx


def get_page_links(driver, url):
    driver.get(url=url)
    links = []
    idx = 0
    max_trial = 10
    trial = 0
    while -1 < idx and trial < max_trial:
        elements = driver.find_elements(By.CSS_SELECTOR, 'a.CardLink')
        for element in elements:
            link = element.get_attribute('href')
            links.append(link)
        idx = pagenation(driver, trial + 2)
        trial += 1
    return links

links = get_page_links(driver, 'https://rapidapi.com/category/Weather')
# links = get_page_links(driver, 'https://rapidapi.com/collection/list-of-free-apis')
len(links)

page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11


240

In [113]:
import re


def to_snake_case(input_string):
    # 문자열 내의 모든 공백과 /를 언더스코어(_)로 대체
    input_string = re.sub(r'[\s/]', '_', input_string)
    # 소문자로 변환
    input_string = input_string.lower()
    # 특수 문자 제거 (예: /)
    input_string = re.sub(r'[^a-z0-9_]', '', input_string)
    # 언더스코어(_) 연속을 하나의 언더스코어로 축약
    input_string = re.sub(r'_+', '_', input_string)
    # 언더스코어(_)로 시작하거나 끝나는 경우 제거
    input_string = input_string.strip('_')
    
    return input_string

# 테스트
input_string = "View from 16 day/{daily} forecast dates"
snake_case_string = to_snake_case(input_string)
print(snake_case_string)  # 출력: "view_from_16_day_daily_forecast_dates"

view_from_16_day_daily_forecast_dates


## 수집한 link에서 자동으로 api subscribe & param 수집

In [139]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import tqdm
import re
import logging


def to_snake_case(input_string):
    # 문자열 내의 모든 공백과 /를 언더스코어(_)로 대체
    input_string = re.sub(r'[\s/]', '_', input_string)
    # 소문자로 변환
    input_string = input_string.lower()
    # 특수 문자 제거 (예: /)
    input_string = re.sub(r'[^a-z0-9_]', '', input_string)
    # 언더스코어(_) 연속을 하나의 언더스코어로 축약
    input_string = re.sub(r'_+', '_', input_string)
    # 언더스코어(_)로 시작하거나 끝나는 경우 제거
    input_string = input_string.strip('_')
    
    return input_string


def replace_by_path_vars(url, sub_path):
    url_paths = url.split("/")
    sub_paths = sub_path.split("/")
    final_paths = []
    for i in range(len(sub_paths) - 1):
        sub_last = sub_paths[-1].strip()
        if sub_last.startswith("{") and sub_last.endswith("}"):
            final_paths.append(sub_last)
        else:
            final_paths.append(url_paths[-1])
        url_paths = url_paths[:-1]
    return "/".join(url_paths + final_paths)


def subscribe_api(driver, link):
    price_link = link + 'pricing'
    driver.get(price_link)
    # elements = driver.find_elements(By.CSS_SELECTOR, 'button.button')
    div_elements = driver.find_elements(By.XPATH, f"//h2[contains(text(), '$0.00')]")
    if 0 < len(div_elements):
        # 부모 HTML 태그 (부모 엘리먼트) 찾기
        buttons = div_elements[0].find_elements(By.XPATH, "./.././/div[@class='buttonContent']")  # ".."은 현재 요소의 부모를 나타내는 XPath입니다
        if 0 < len(buttons):
            buttons[0].click()
            subscribe = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.XPATH, f"//button[@aria-label='CreateSubscriptionBtn']"))
            )
            subscribe.click()
            complete = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.XPATH, f"//h2[contains(text(), 'Subscription Created Successfully')]"))
            )
            print(price_link, complete.text)
        return True
    else:
        return False


def check_api_response(driver):
    response = None
    test_button = WebDriverWait(driver, 2).until(
        EC.presence_of_element_located((By.XPATH, f".//button[@data-id='endpoint-test-button']"))
    )
    if test_button:
        test_button.click()
        response_span = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, f".//div[@data-id='response-status']/span[@class='ant-tag ant-tag-has-color']"))
        )
        response = response_span.text
        code_snippets = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, f".//div[contains(text(), 'Code Snippets')]"))
        )
        code_snippets.click()
    return response
    
            
def get_api_params(driver, api_name, check_api=False):
    frame = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, f"//iframe[@class='PlaygroundContainer']"))
    )
    driver.get(url=frame.get_attribute("src"))
    api_list = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'sc-eWhHU')]"))
    )    
    api_elements = driver.find_elements(By.XPATH, f"//div[contains(@class, 'sc-eWhHU')]")
    api_list = []
    for api in api_elements:
        root_url = None
        url = None
        func_name = api.get_attribute("title")
        method = api.find_element(By.XPATH, ".//span[@class='notranslate']").text
        button = api.find_element(By.XPATH, ".//div[@role='button']")
        button.click()
        
        response = None
        if check_api:
            response = check_api_response(driver)
                
        required_parameters = []
        optional_parameters = []
        api_params = driver.find_elements(By.XPATH, ".//div[@class='ant-collapse-content-box']/div[contains(@class, 'sc-gCkVGe')]")
        for api_param in api_params:
            name = api_param.find_element(By.XPATH, ".//label[@class='name']").text
            if (not ("RapidAPI" in name)) and name != "all":
                data_type = api_param.find_element(By.XPATH, ".//div[@class='type']").text
                extra = api_param.find_element(By.XPATH, ".//div[@class='extra']").text
                description = api_param.find_element(By.XPATH, ".//div[@class='description']").text
                meta_fields = api_param.find_elements(By.XPATH, f".//div[@class='meta-fields']/p")
                input_value = None
                input_values = api_param.find_elements(By.XPATH, ".//input[@type='text']")
                if 0 < len(input_values):
                    input_value = input_values[0].get_attribute("value")
                metas = []
                for meta in meta_fields:
                    metas.append(meta.text)
                if extra.startswith("REQUIRED"):
                    target_list = required_parameters
                else:
                    target_list = optional_parameters
                extra = extra.split("\n")
                if 1 < len(extra):
                    extra = extra[1]
                else:
                    extra = ""
                api_info = dict(
                    name=name,
                    type=data_type,
                    input_value=input_value,
                    extra=extra,
                    metas=metas,
                    description=description,
                )
                target_list.append(api_info)
            elif "RapidAPI-Host" in name:
                root_url = api_param.find_element(By.XPATH, ".//input[@id='x-rapidapi-host']").get_attribute("value")
        if root_url:
            url = driver.find_element(By.XPATH, f".//span[contains(text(), '" + root_url + "')]")
            url = url.text
            if url.startswith("'") or url.startswith("\""):
                url = url[1:]
            if url.endswith("'") or url.startswith("\""):
                url = url[:-1]
            url = replace_by_path_vars(url, func_name)
            split_urls = url.split(root_url)
            func_url = split_urls[-1]
        func_name = to_snake_case(f"{api_name} {func_name}")
        info = dict(
                root_url=root_url,
                func_url=func_url, 
                func_name=func_name,
                method=method, 
                required_parameters=required_parameters, 
                optional_parameters=optional_parameters,
                response=response,
               )
        api_list.append(info)
    return api_list


def get_api(driver, link, check_api=False):
    driver.get(link)
    tool_description = driver.find_element(By.XPATH, ".//div[@class='MarkdownPreview']/p").text
    name = driver.find_element(By.XPATH, ".//div[@class='ProductInfo']/div[@class='Name']/h2").text
    metas = driver.find_elements(By.XPATH, ".//div[@class='ProductInfo']/div[@class='About']/a")
    cate = metas[1].text
    api_list = get_api_params(driver, name, check_api)
    return dict(
        name=name,
        cate=cate,
        tool_description=tool_description,
        api_list=api_list,
    )


def visit_links(links, subscribe=True, collect_api=True, from_idx=0, size=None, 
                show=False, check_api=False):
    if size is None:
        size = len(links)
    collect_apis = []
    for link in tqdm.tqdm(links[from_idx:from_idx + size]):
        try:
            if subscribe:
                is_success = subscribe_api(driver, link)
            else:
                is_success = True
            if is_success and collect_api:
                collected = get_api(driver, link, check_api=check_api)
                if show:
                    print(collected)
                collect_apis.append(collected) 
        except Exception as e:
            logging.error(link, exc_info=True)
    return collect_apis


# links = get_page_links(driver, 'https://rapidapi.com/collection/city-data-api')
# links = get_page_links(driver, 'https://rapidapi.com/category/Weather')
# links = get_page_links(driver, 'https://rapidapi.com/collection/trending-news-apis')
api_info = visit_links(links, subscribe=True, collect_api=True, from_idx=0, size=None, 
                       show=True, check_api=True)
len(api_info)

  0%|                                                                                                                                                                                                                                                                                           | 0/240 [00:00<?, ?it/s]ERROR:root:https://rapidapi.com/darkskyapis/api/dark-sky/
Traceback (most recent call last):
  File "/var/folders/rp/wmsf2gtd4ddg9wfz3b3_mn9c0000gn/T/ipykernel_51910/2889434308.py", line 184, in visit_links
    is_success = subscribe_api(driver, link)
  File "/var/folders/rp/wmsf2gtd4ddg9wfz3b3_mn9c0000gn/T/ipykernel_51910/2889434308.py", line 51, in subscribe_api
    complete = WebDriverWait(driver, 2).until(
  File "/Users/stepping1st/miniforge3/lib/python3.9/site-packages/selenium/webdriver/support/wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
0   chromedriver              

{'name': 'Visual Crossing Weather', 'cate': 'Weather', 'tool_description': 'Visual Crossing Weather API provides instant access to both historical weather records and weather f', 'api_list': [{'root_url': 'visual-crossing-weather.p.rapidapi.com', 'func_url': '/forecast', 'func_name': 'visual_crossing_weather_weather_forecast_data', 'method': 'GET', 'required_parameters': [{'name': 'aggregateHours', 'type': 'NUMBER', 'input_value': None, 'extra': 'The interval between weather forecast data in the output. 1 represents an hourly forecast, 24 represents a daily forecast. As the source data is calculated at the hourly level, records calculated at 12 or 24 hours are aggregated to indicate the predominant weather condition during that time period. For example the maximum temperature, total precipitation, maximum windspeed etc. Supported values 1,12 or 24.', 'metas': [], 'description': 'The interval between weather forecast data in the output. 1 represents an hourly forecast, 24 represents a d

  2%|██████▊                                                                                                                                                                                                                                                                          | 6/240 [02:59<3:09:06, 48.49s/it]

{'name': 'WeatherAPI.com', 'cate': 'Weather', 'tool_description': 'WeatherAPI.com is a powerful fully managed free weather and geolocation API provider that provides e', 'api_list': [{'root_url': 'weatherapi-com.p.rapidapi.com', 'func_url': '/current.json', 'func_name': 'weatherapicom_realtime_weather_api', 'method': 'GET', 'required_parameters': [{'name': 'q', 'type': 'STRING', 'input_value': '53.1,-0.13', 'extra': 'Query parameter based on which data is sent back. It could be following: Latitude and Longitude (Decimal degree) e.g: q=48.8567,2.3508 city name e.g.: q=Paris US zip e.g.: q=10001 UK postcode e.g: q=SW1 Canada postal code e.g: q=G2J metar: e.g: q=metar:EGLL iata:<3 digit airport code> e.g: q=iata:DXB auto:ip IP lookup e.g: q=auto:ip IP address (IPv4 and IPv6 supported) e.g: q=100.0.0.1', 'metas': [], 'description': 'Query parameter based on which data is sent back. It could be following: Latitude and Longitude (Decimal degree) e.g: q=48.8567,2.3508 city name e.g.: q=Paris 

  3%|███████▉                                                                                                                                                                                                                                                                         | 7/240 [03:27<2:41:54, 41.69s/it]

...

{'name': 'World Weather Online API', 'cate': 'Weather', 'tool_description': 'Free Weather API and Geo API for worldwide locations, hourly weather, 14-day weather, historical wea', 'api_list': [{'root_url': 'world-weather-online-api1.p.rapidapi.com', 'func_url': '/weather.ashx', 'func_name': 'world_weather_online_api_local_weather_api', 'method': 'GET', 'required_parameters': [{'name': 'q', 'type': 'STRING', 'input_value': '40.73,-73', 'extra': '', 'metas': [], 'description': ''}], 'optional_parameters': [{'name': 'num_of_days', 'type': 'NUMBER', 'input_value': None, 'extra': '', 'metas': [], 'description': ''}, {'name': 'tp', 'type': 'ENUM', 'input_value': None, 'extra': '', 'metas': [], 'description': ''}, {'name': 'lang', 'type': 'STRING', 'input_value': 'en', 'extra': '', 'metas': [], 'description': ''}, {'name': 'aqi', 'type': 'STRING', 'input_value': 'yes', 'extra': '', 'metas': [], 'description': ''}, {'name': 'alerts', 'type': 'STRING', 'input_value': 'no', 'extra': '', 'metas':

ERROR:root:https://rapidapi.com/aptitudeapps/api/koppen-climate-classification/
Traceback (most recent call last):
  File "/var/folders/rp/wmsf2gtd4ddg9wfz3b3_mn9c0000gn/T/ipykernel_51910/2889434308.py", line 188, in visit_links
    collected = get_api(driver, link, check_api=check_api)
  File "/var/folders/rp/wmsf2gtd4ddg9wfz3b3_mn9c0000gn/T/ipykernel_51910/2889434308.py", line 167, in get_api
    api_list = get_api_params(driver, name, check_api)
  File "/var/folders/rp/wmsf2gtd4ddg9wfz3b3_mn9c0000gn/T/ipykernel_51910/2889434308.py", line 98, in get_api_params
    response = check_api_response(driver)
  File "/var/folders/rp/wmsf2gtd4ddg9wfz3b3_mn9c0000gn/T/ipykernel_51910/2889434308.py", line 67, in check_api_response
    response_span = WebDriverWait(driver, 3).until(
  File "/Users/stepping1st/miniforge3/lib/python3.9/site-packages/selenium/webdriver/support/wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutExcept

{'name': 'Air Quality by API-Ninjas', 'cate': 'Weather', 'tool_description': 'Get current air quality data for any region. See more info at See more info at https://api-ninjas.co', 'api_list': [{'root_url': 'air-quality-by-api-ninjas.p.rapidapi.com', 'func_url': '/airquality/v1', 'func_name': 'air_quality_by_apininjas_v1_airquality', 'method': 'GET', 'required_parameters': [], 'optional_parameters': [{'name': 'lat', 'type': 'NUMBER', 'input_value': None, 'extra': 'Latitude of desired location. If used, the lon parameter must also be supplied.', 'metas': [], 'description': 'Latitude of desired location. If used, the lon parameter must also be supplied.'}, {'name': 'lon', 'type': 'NUMBER', 'input_value': None, 'extra': 'Longitude of desired location. If used, the lat parameter must also be supplied.', 'metas': [], 'description': 'Longitude of desired location. If used, the lat parameter must also be supplied.'}, {'name': 'city', 'type': 'STRING', 'input_value': 'Seattle', 'extra': 'City 

ERROR:root:https://rapidapi.com/apidojo/api/weather338/
Traceback (most recent call last):
  File "/var/folders/rp/wmsf2gtd4ddg9wfz3b3_mn9c0000gn/T/ipykernel_51910/2889434308.py", line 188, in visit_links
    collected = get_api(driver, link, check_api=check_api)
  File "/var/folders/rp/wmsf2gtd4ddg9wfz3b3_mn9c0000gn/T/ipykernel_51910/2889434308.py", line 167, in get_api
    api_list = get_api_params(driver, name, check_api)
  File "/var/folders/rp/wmsf2gtd4ddg9wfz3b3_mn9c0000gn/T/ipykernel_51910/2889434308.py", line 98, in get_api_params
    response = check_api_response(driver)
  File "/var/folders/rp/wmsf2gtd4ddg9wfz3b3_mn9c0000gn/T/ipykernel_51910/2889434308.py", line 67, in check_api_response
    response_span = WebDriverWait(driver, 3).until(
  File "/Users/stepping1st/miniforge3/lib/python3.9/site-packages/selenium/webdriver/support/wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrac

13

# 

### 응답이 200 온것만 필터링

In [141]:
import copy


def filter_by_response(apis):
    result = []
    for api in apis:
        api_list = [a for a in api["api_list"] if a["response"] == "200"]
        if 0 < len(api_list):
            api = copy.copy(api)
            api["api_list"] = api_list
            result.append(api)
    return result


filtered_api = filter_by_response(api_info)
len(filtered_api)

12

### 수집한 데이터 저장

In [156]:
import json

with open('weather.jsonl', 'w') as file:
    for api in filtered_api:
        file.write(json.dumps(api))
        file.write("\n")

In [154]:
for api in filtered_api:
    print(api["name"])

Visual Crossing Weather
WeatherAPI.com
Open Weather
AI Weather by Meteosource
Forecast
Easy Weather
Weather API - By Any City
Plant Hardiness Zone
Sunrise Sunset Times
Weather by API-Ninjas
World Weather Online API
Air Quality by API-Ninjas
